# Text Analysis for Central Bank Communication

## Initial steps

In [ ]:
# Initial imports

import numpy as np
import pandas as pd
import gensim
import os
import io
import logging
from matplotlib import pyplot as plt
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
!pip install pyLDAvis

In [ ]:
from google.colab import files
uploaded = files.upload() # use widget to upload your file

In [ ]:
# Import data
df = pd.read_csv(io.StringIO(uploaded['FOMCtext.csv'].decode('utf-8')))

In [ ]:
df.head()

This dataframe doesn't look very nice, lets clean it up first

In [ ]:
# Clean up dataframe
df = df.loc[:, ['Date', 'Text']] # only want these columns
df.loc[:, 'PandasDateTime'] = pd.to_datetime(df.loc[:, 'Date'], format='S_%Y%m%d.txt') # create a proper datetime column

In [ ]:
df.head()

## Pre-processing text

Text data is often (always?) messy. Lets take a look at the first entry

In [ ]:
df.Text[0]

"Release Date: February 4, 1994\n \n\nFor immediate release\nChairman Alan Greenspan announced today that the Federal Open Market Committee decided to increase slightly the degree of pressure on reserve positions. The action is expected to be associated with a small increase in short-term money market interest rates.\n\nThe decision was taken to move toward a less accommodative stance in monetary policy in order to sustain and enhance the economic expansion.\n\nChairman Greenspan decided to announce this action immediately so as to avoid any misunderstanding of the Committee's purposes, given the fact that this is the first firming of reserve market conditions by the Committee since early 1989.\n\n"

What things might we want to strip out of this text to make it less noisy data?

Gensim provides an easy way for us to preprocess the text before we do any analysis on it. Below we define a function that will take in the raw text column from our dataframe, and spit out a set of processed documents. The cell below imports the different options you have for preprocessing the text. You should decide which functions you want to add to your preprocessing function.

Furthermore, you should use the lambda functions to replace specific parts of the text that you want to modify.


In [ ]:
# Import gensim preprocessing functions
from gensim.parsing.preprocessing import preprocess_string # overall function that will allow us to do our preprocessing

#
from gensim.parsing.preprocessing import remove_stopwords # this removes stopwords like 'and' and 'the'
from gensim.parsing.preprocessing import stem_text # this stems text to its root, so economics and econmist both become economi
from gensim.parsing.preprocessing import strip_punctuation # this removes punctuation
from gensim.parsing.preprocessing import strip_short # this removes words shorter than a length the user provides (default 3)
from gensim.parsing.preprocessing import strip_numeric # this removes numbers

In [ ]:
# Define a function that preprocesses the text

def Prep(docseries):
    '''Function that does some preprocessing
    INPUTS: docseries - pandas series, each element is one long string not broken up by tokens
    OUTPUTS: result - list of lists, a list of documents each with a list of token strings'''
    result=[]
    filters = [lambda x: x.replace('date', ''), # fill in other lambda functions
               # fill
               # fill
               #
               # fill in other gensim processing functions
               ]
    result = [preprocess_string(document, filters=filters) for document in docseries.tolist()]
    return result

In [ ]:
# Use your function to preprocess the text
df['PreppedText'] = Prep(df.Text)

In [ ]:
# Look at the new dataframe
df.head()

In [ ]:
# Look at the new processed text
df.PreppedText[0]

## Dictionary methods

We can just count words if we want a measure of what the FOMC is talking about. Try 'alan', 'asset', 'basis' etc etc

In [ ]:
df.loc[:, 'word_count'] = df.loc[:, 'PreppedText'].apply(lambda x: x.count('asset')) # 

In [ ]:
# Plot results

plt.plot(df.PandasDateTime, df.word_count)
plt.show()

This is probably OK to use now. But we might be able to do better. Want we are trying to do is increase the signal:noise ratio in the text. A lot of what we just did is remove noise. E.g. fullstops are not useful to us, and so keeping them in adds noise to the dataset, so we removed them. 

But now it looks like there are words that are going to be extremely common across all FOMC statements. Words like 'release', if you didn't remove them in preprocessing, occur in every statement. And words like 'monetary' probably occur in nearly all of them. These words aren't going to tell us anything because there is no variation in them between statements. Let's try to remove them.


In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(df.PreppedText.to_list()) # be clear on what types of objects your function takes

# Filter out words that occur less than 3 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=# fill in, no_above= # fill in)

2021-09-09 15:52:51,192 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-09-09 15:52:51,228 : INFO : built Dictionary(1317 unique tokens: ['1989', '1994', 'accommod', 'action', 'alan']...) from 147 documents (total 31305 corpus positions)
2021-09-09 15:52:51,232 : INFO : discarding 362 tokens: [('1989', 1), ('action', 139), ('avoid', 1), ('chairman', 115), ('committe', 146), ('decid', 123), ('econom', 141), ('expect', 105), ('fact', 1), ('feder', 147)]...
2021-09-09 15:52:51,234 : INFO : keeping 955 tokens which were in no less than 2 and no more than 102 (=70.0%) documents
2021-09-09 15:52:51,238 : INFO : resulting dictionary: Dictionary(955 unique tokens: ['1994', 'accommod', 'alan', 'announc', 'associ']...)


In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in df.PreppedText.to_list()]

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

## LDA

LDA is a generative model. In other words LDA assumes a specific mechanism through which each paragraph is created:

• For each topic (k = 1, 2...K), independently draw a distribution over words ($β_k$) from a Dirichlet($η$)
distribution

• For each paragraph (p = 1, 2...P), independently draw a distribution over topics ($θ_p$) from a
Dirichlet($α$) distribution

• Then for each word in a given paragraph ($w_n$,p):

– Randomly draw a topic ($z_n$,p) from the topic distribution assigned to that paragraph ($θ_p$)

– Randomly draw a word from that topic’s distribution of words ($β_zn$,p)

This process defines a probability distribution over the data that we observe. Then our problem is to compute the posterior distributions of words for each topic ($β_k$ for all k), and the topic distribution for every paragraph ($θ_p$ for all p), given the words we observe.

In [ ]:
# Train LDA model
from gensim.models import LdaModel

# Set training parameters.
num_topics =  # number of topics
chunksize = # how many docs to put in RAM
passes = # training epochs
iterations = 800
eval_every = 0

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

lda_viz = gensimvis.prepare(model, corpus, dictionary)

In [ ]:
# Do visualisation
lda_viz

## Word2Vec

Sometimes we are not so interested in just reducing dimensionality. We might want to find words similar to another word, for example. Tasks like these require us to retain the relationships between words (i.e. we cannot throw that information away by using Bag of Words models).

Neural network architectures for NLP are now common. We will review a famous one, word2vec.

Word2vec (in our case) does two things:
1. Asks a shallow neural network topredict a target word, given the context of the word (i.e. the words in a small window around the target word)
2. After training the network, the weights corresponding to the hidden layer are extracted and used as vector representations of words

In [ ]:
w2vmodel = gensim.models.Word2Vec(df.PreppedText,
                               min_count=,        # ignore words with frequency less than this
                               window = ,  # window size around word to predict
                               size=,    # length of vector # new version: vector_size
                               workers=,          # number of cores to use
                               iter=,          # number of times to iterate over the corpus # new version: epochs
                               negative=5)  # negative sub sampling parameter

In [ ]:
# Does it work?
w2vmodel.wv.most_similar('alan')